# Table Operations

_This is a Jupyter Notebook file, make sure to run all the cells in the presented order to avoid any errors_

This tutorial details how to retrieve and modify data from ETABS model DataBase Tables using `PyCSI`. This example uses an ETABS model; however, connecting to a SAP2000 or SAFE model is done by changing only the ETABSmodel class for the desired software.

## Opening example model

See the [Basic Functions](2_Basic_functions.ipynb) tutorial for details of the following code

In [ ]:
# Python packages imports
import os
import sys

###################################################################
# The following lines are only necessary for these tutorials
from pathlib import Path
pyCSI_dir = str(Path(os.getcwd()).parents[2])
os.chdir((pyCSI_dir))
###################################################################

# PyCSI import
import pyCSI

# example_model location 
# DO NOT MODIFY THE FOLLOWING LINES
VERSION = 21
EXAMPLE_LOCATION = os.sep.join([pyCSI_dir, 'docs', 'tutorials', 'resources', 'example_model'])
EXAMPLE_NAME = 'pyCSI_example - v' + str(VERSION) + '.EDB'
EXAMPLE_FILE = os.sep.join([EXAMPLE_LOCATION, EXAMPLE_NAME])
open_file = False  # Flag to be used in the next step

model = pyCSI.ETABSModel(version=VERSION)
try:
    # Calling get_model() with no arguments connects to an active model
    model.get_model()
    file_name = model.get_file_name()  # Get the name of the connected model

    # Check that the connected model is the pyCSI_example - v21.EDB file
    # If not, set the flag to open the model in the next step
    if not file_name == EXAMPLE_NAME:
        print('Not connected to Example Model')
        open_file = True
    
except pyCSI.APIConnectionError:
    # If no active model is found, an Attribute error is raised
    # Set the flag to open the model in the next step
    print('No active model found')
    open_file = True

if open_file:
    print('Opening example model')
    model.get_model(active_model=False, file_location=EXAMPLE_FILE)

## Table property

`PyCSI` provides the `.tables` property to access all the DataBase Table methods. The following sections detail the available methods.  

## Get available tables in the model

ETABS has a large number of tables, each of which has a unique Table Key Identifier.

The `.tables.get_available_tables()` method allows one to get a list that contains the Table Keys of all the available tables in the model. You can use these keys to access the table data.

**Note**: This method list the tables that contain data at the moment of the request. This means that if no data exists for a specific table, for example the "Beam Forces" when the model is not run, then this Table Key will not appear in the returned list.

In [ ]:
# Unlock the model
model.lock = False

# Available tables when model is unlocked
table_names = model.tables.get_available_tables()
print(f'Found {len(table_names)} available tables when model is unlocked', end='\n\n')

# Start analysis
model.analysis.run_analysis()

# Available tables when model is locked
table_names = model.tables.get_available_tables()
print(f'Found {len(table_names)} available tables when model is run', end='\n\n')

# Some of the available tables
n = 20
print(f'First {n} available tables')
print(*table_names[:n], sep='\n', end='\n\n')

## Accessing table data

You can access the data of a DataBase table only by providing its Table Key.

Use the `.tables.get_table_dataframe(table_key)` to retrieve the data of a table. The data is provided in a Pandas DataFrame format.

In [ ]:
TABLE_KEY = 'Story Forces'
story_definitions = model.tables.get_table_dataframe(TABLE_KEY)
print(TABLE_KEY)
display(story_definitions)


### Getting all fields in the table

By default, ETABS omits columns with no data in the table. For example, The 'Step Type' column on the Story Forces table is not provided if there is no Response Spectrum Load Case.

To get all the headers in the table, regardless of its content, provide the argument `include_all_headers = True` to the `.tables.get_table_dataframe()` method.

In [ ]:
TABLE_KEY = 'Story Forces'
story_definitions = model.tables.get_table_dataframe(TABLE_KEY, include_all_headers=True)
print(TABLE_KEY)
display(story_definitions)


### Get data from a specific group

When using ETABS, you can get table results only for the objects you have selected at the moment. You can achieve a similar behavior by providing the name of an ETABS group in the `group` argument. Doing this, the returned table will contain only the data for the objects in the specified group.

The example model has a defined group, `Roof cols GL 4`, which contains only the columns at grid line 4 located at the Roof level. The following code gets the frame properties for these specific columns.


In [ ]:
TABLE_KEY = 'Frame Assignments - Summary'
frame_assignments = model.tables.get_table_dataframe(TABLE_KEY, group='Roof cols GL 4')
print(TABLE_KEY)
display(frame_assignments)


### Get data from specific load patterns, load cases and load combos

ETABS gives the option to get table data for specific Load Patterns (LPs), Load Cases (LCs) and Load Combinations (LCombos) through its 'Show Tables' format.

<img src="..\..\images\tutorials\show_tables.png" alt="Show Tables Format" width="600"/>

In `PyCSI`, you can achieve this by modifying the `.tables.load_patterns`, `.tables.load_cases` or `.tables.load_combos` properties.

These properties accept the following values:
* A list containing the names of the LPs, LCs or LCombos to be included in the tables
* 'all' to select all defined LPs, LCs or LCombos
* None to not include any LP, LC or LCombo

In [ ]:
model.tables.load_patterns = 'all'  # Include all load patterns
print(f'{model.tables.load_patterns=}', end='\n\n')

model.tables.load_cases = None  # Do not include any load case in the table
print(f'{model.tables.load_cases=}', end='\n\n')

model.tables.load_combos = ['[ASD 2] D+L', '[LRFD 1] 1.4D']  # Include only specified load combinations
print(f'{model.tables.load_combos=}', end='\n\n')



All subsequent calls to .tables.get_table_dataframe() after assigning a value to the .load_patterns, .load_cases or .load_combinations properties will contain the specified LPs, LCs and LCombos.

In [ ]:
TABLE = 'Story Forces'
story_forces = model.tables.get_table_dataframe(TABLE)
print(TABLE_KEY)
display(story_forces)


## Edit DataBase Tables

**Coming soon...**

## Next -> [Group Objects](4_Group_Methods.ipynb)

**REMEMBER to shut down the Kernel before leaving**

Click `Kernel -> Shut Down Kernel`

### Contact

For questions or comments please reach out to:

* Luis Pancardo: [lpancardo@degenkolb.com](lpancardo@degenkolb.com)<br/>
* Daniel Gaspar:  [dgaspar@degenkolb.com](dgaspar@degenkolb.com)<br/>
